In [1]:
import requests
import json
import csv
import pandas as pd
from itertools import chain
from collections import Counter

In [2]:
tmdb_api_key = 'c0228e59511475167a1c003c52f3fc21'

In [3]:
def get_movie_data(movie_id, api_key):
    base_url = 'https://api.themoviedb.org/3/movie/'
    url = f"{base_url}{movie_id}?api_key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None


In [4]:
# FUNCTION TO GET KEYWORDS + ACTORS + DIRECTORS 

import requests

def get_movie_keywords_and_credits(movie_id, api_key):
    base_url = 'https://api.themoviedb.org/3/movie/'
    
    # Get keywords
    keyword_url = f"{base_url}{movie_id}/keywords?api_key={api_key}"
    keyword_response = requests.get(keyword_url)
    
    # Get credits (cast and crew)
    credit_url = f"{base_url}{movie_id}/credits?api_key={api_key}"
    credit_response = requests.get(credit_url)
    
    if keyword_response.status_code == 200 and credit_response.status_code == 200:
        keywords = keyword_response.json()
        credits = credit_response.json()
        
        # Get directors
        directors = [crew_member for crew_member in credits['crew'] if crew_member['job'] == 'Director']
        
        # Get actors (cast)
        actors = credits['cast']
        
        return {
            'keywords': keywords,
            'directors': directors,
            'actors': actors
        }
    else:
        return None


In [5]:
def extract_fields(data, movie_data):
    fields = ['title', 'release_date', 'original_language', 'runtime', 'genres', 'production_companies', 'popularity', 'vote_count', 'vote_average', 'budget', 'revenue']
    extracted_data = {field: data.get(field, None) for field in fields}
    
    # Extract keywords
    extracted_data['keywords'] = [keyword['name'] for keyword in movie_data['keywords']['keywords']]
    
    # Extract directors
    extracted_data['directors'] = [director['name'] for director in movie_data['directors']]
    
    # Extract actors (cast)
    extracted_data['actors'] = [actor['name'] for actor in movie_data['actors']]
    
    return extracted_data


In [6]:
import time

movie_data = []
movie_count = 0
max_movies = 7000

# Important to add a sleep duration to prevent overwhelming the site
sleep_duration = 0.1

for movie_id in range(1, 400000):
    if movie_count < max_movies:
        data = get_movie_data(movie_id, tmdb_api_key)

        if data:
            # Check if the movie is not adult and has a value for budget and revenue
            if not data.get('adult', False) and data.get('budget', 0) > 0 and data.get('revenue', 0) > 0:
                keywords = get_movie_keywords_and_credits(movie_id, tmdb_api_key)
                if keywords:
                    extracted_data = extract_fields(data, keywords)
                    movie_data.append(extracted_data)
                    movie_count += 1

                    # Print progress update every 100 movies
                    if movie_count % 10 == 0:
                        print(f"Processed {movie_count} movies")

        time.sleep(sleep_duration)  # Sleep for the specified duration between requests
    else:
        break


Processed 10 movies
Processed 20 movies
Processed 30 movies
Processed 40 movies
Processed 50 movies
Processed 60 movies
Processed 70 movies
Processed 80 movies
Processed 90 movies
Processed 100 movies
Processed 110 movies
Processed 120 movies
Processed 130 movies
Processed 140 movies
Processed 150 movies
Processed 160 movies
Processed 170 movies
Processed 180 movies
Processed 190 movies
Processed 200 movies
Processed 210 movies
Processed 220 movies
Processed 230 movies
Processed 240 movies
Processed 250 movies
Processed 260 movies
Processed 270 movies
Processed 280 movies
Processed 290 movies
Processed 300 movies
Processed 310 movies
Processed 320 movies
Processed 330 movies
Processed 340 movies
Processed 350 movies
Processed 360 movies
Processed 370 movies
Processed 380 movies
Processed 390 movies
Processed 400 movies
Processed 410 movies
Processed 420 movies
Processed 430 movies
Processed 440 movies
Processed 450 movies
Processed 460 movies
Processed 470 movies
Processed 480 movies
P

Processed 3790 movies
Processed 3800 movies
Processed 3810 movies
Processed 3820 movies
Processed 3830 movies
Processed 3840 movies
Processed 3850 movies
Processed 3860 movies
Processed 3870 movies
Processed 3880 movies
Processed 3890 movies
Processed 3900 movies
Processed 3910 movies
Processed 3920 movies
Processed 3930 movies
Processed 3940 movies
Processed 3950 movies
Processed 3960 movies
Processed 3970 movies
Processed 3980 movies
Processed 3990 movies
Processed 4000 movies
Processed 4010 movies
Processed 4020 movies
Processed 4030 movies
Processed 4040 movies
Processed 4050 movies
Processed 4060 movies
Processed 4070 movies
Processed 4080 movies
Processed 4090 movies
Processed 4100 movies
Processed 4110 movies
Processed 4120 movies
Processed 4130 movies
Processed 4140 movies
Processed 4150 movies
Processed 4160 movies
Processed 4170 movies
Processed 4180 movies
Processed 4190 movies
Processed 4200 movies
Processed 4210 movies
Processed 4220 movies
Processed 4230 movies
Processed 

NameError: name 'column_names' is not defined

In [11]:
# SHOW TABLE 

column_names = ['title', 'release_date', 'original_language', 'runtime', 'genres', 'actors', 'directors', 'production_companies', 'keywords', 'popularity', 'vote_count', 'vote_average', 'budget', 'revenue']

movie_data_df = pd.DataFrame(movie_data, columns=column_names)
pd.set_option('display.max_colwidth', 13)
display(movie_data_df)


,title,release_date,original_language,runtime,genres,actors,directors,production_companies,keywords,popularity,vote_count,vote_average,budget,revenue
0,Four Rooms,1995-12-09,en,98,[{'id': 8...,[Tim Roth...,[Allison ...,[{'id': 1...,"[hotel, n...",21.376,2357,5.758,4000000,4257354
1,Judgment ...,1993-10-15,en,109,[{'id': 2...,[Emilio E...,[Stephen ...,[{'id': 1...,"[chicago,...",7.675,284,6.556,21000000,12136938
2,Star Wars,1977-05-25,en,121,[{'id': 1...,[Mark Ham...,[George L...,[{'id': 1...,"[android,...",120.016,18563,8.208,11000000,775398007
3,Finding Nemo,2003-05-30,en,100,[{'id': 1...,[Albert B...,[Andrew S...,[{'id': 3...,[parent c...,74.931,17613,7.823,94000000,940335536
4,Forrest Gump,1994-06-23,en,142,[{'id': 3...,[Tom Hank...,[Robert Z...,[{'id': 4...,[vietnam ...,52.131,24570,8.481,55000000,677387716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,Jackass P...,2013-10-22,en,92,[{'id': 3...,[Johnny K...,[Jeff Tre...,[{'id': 4...,[duringcr...,14.690,1407,6.104,15000000,151831537
6996,Forbidden...,2014-01-30,ru,128,[{'id': 5...,[Jason Fl...,[Oleg Ste...,[{'id': 7...,"[monster,...",9.949,252,5.900,26000000,34592118
6997,Batman v ...,2016-03-23,en,152,[{'id': 2...,[Ben Affl...,[Zack Sny...,[{'id': 1...,[vigilant...,65.151,16721,5.900,250000000,873637528
6998,Zombie Br...,1986-05-22,en,92,[{'id': 2...,[],[Carmelo ...,[],"[gore, zo...",1.129,7,2.500,10,15


In [12]:
# Save the DataFrame to a CSV file
movie_data_df.to_csv('movies_data.csv', index=False, encoding='utf-8')

In [1]:
movie_data_df.dtypes

### End of initial data collection. Proceed to cleaning and encoding. 

NameError: name 'movie_data_df' is not defined